In [ ]:
# Housekeeping

## Library imports

In [7]:
#TODO: use %store mydf and %store -r mydf (and also del in the first notebook) to divide the processes in the separate notebooks

if False:
    import sys
    !{sys.executable} -m pip install -r requirements.txt

In [ ]:
# Core Libraries
import pandas as pd
import seaborn as sns
from colorama import Fore, Style
import warnings
import matplotlib.pyplot as plt
import numpy as np
import re
import os

# Machine Learning - Core
import sklearn

# Statistical & Data Processing
from scipy import stats
from sklearn.model_selection import GroupShuffleSplit
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Machine Learning - Models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Model Interpretability
from sklearn.metrics import mean_squared_error, r2_score
from supertree import SuperTree

#Features selection
import shap

# Custom Functions
from functions import (
    random_forest_benchmark,
    xgboost_benchmark,
    adaboost_benchmark,
    gradient_boosting_benchmark,
    lightgbm_benchmark,
#    nn_feature_search,
    explain_with_lime,
    explain_with_shap,
    cross_validate_feature_cutoffs,
    plot_feature_cutoff_comparison,
    plot_model_comparison_heatmap,
    plot_residuals_analysis,
    plot_prediction_intervals,
    plot_learning_curves,
    feature_selection_pipeline,
    final_battle,
 #   set_global_seeds,
    get_taxonomic_level,
    filter_features_by_level,
    stochastic_gradient_boosting_benchmark,
    kernel_random_forest_benchmark,
    compare_feature_selection_methods,
    find_best_evaluation_metric
)

#Importing Master Seed 3004
from functions import MASTER_SEED

## Settings

In [11]:
warnings.filterwarnings("ignore")
sklearn.set_config(transform_output="pandas")
print(Style.RESET_ALL)

'''gpus = tf.config.list_physical_devices('GPU')
if gpus or False: #<- Change to True if you want to torture your computer (:
    RUN_NN = True
    print("GPU found")
else:
    RUN_NN = False'''

'gpus = tf.config.list_physical_devices(\'GPU\')\nif gpus or False: #<- Change to True if you want to torture your computer (:\n    RUN_NN = True\n    print("GPU found")\nelse:\n    RUN_NN = False'

In [12]:
# Set global random seeds for reproducibility
set_global_seeds(MASTER_SEED)

NameError: name 'set_global_seeds' is not defined

## Data imports
Data was manually edited, to convert the mpa411.txt TSV format to a CSV format. Otherwise, Pandas was loading it as a single column, somehow. The first row, containing only "#mpa_vJun23_CHOCOPhlAnSGB_202403" was removed.

In [ ]:
data = pd.read_csv('../data/raw/MAI3004_lucki_mpa411.csv')
metadata = pd.read_csv('../data/raw/MAI3004_lucki_metadata_safe.csv')
print(
    f"Data successfully imported. \n shape of data: {data.shape} \n "
    f"Shape of metadata: {metadata.shape}"
)

assert data.shape == (6903, 932), "Data has the wrong shape. Check the CSV formatting."
assert metadata.shape == (930, 6), "Metadata has the wrong shape. Check the CSV formatting."

# Data preprocessing


## Merge data and metadata


In [ ]:
sample_cols = [col for col in data.columns if col.startswith("mpa411_")]

sample_abundances = (
    data[['clade_name'] + sample_cols]
    .set_index('clade_name')
    .transpose()
    .rename_axis('original_sample_id')
    .reset_index()
    .rename(columns={'original_sample_id': 'sample_id'})
)

sample_abundances["sample_id"] = (
    sample_abundances["sample_id"].str.removeprefix(
        "mpa411_",
    )
)

metadata_common = metadata[
    metadata["sample_id"].isin(sample_abundances["sample_id"])
].copy()
merged_samples = metadata_common.merge(
    sample_abundances,
    on="sample_id",
    how="inner",
)

merged_samples.drop(columns=['year_of_birth', 'body_product'], inplace=True)

print(f"Metadata rows (original): {metadata.shape[0]}")
print(f"Metadata rows with matching samples: {metadata_common.shape[0]}")
print(
    f"Metadata rows without matching samples: "
    f"{metadata_common.shape[0]-metadata_common.shape[0]}"
)
print(f"Merged dataframe shape: {merged_samples.shape}")

In [ ]:
merged_samples.head()

## Encoding

In [ ]:
# Sex and family_ID
encoded_samples = merged_samples.copy().dropna(subset="age_group_at_sample")

encoded_samples["sex"] = (
    encoded_samples["sex"]
    .fillna("unknown")
    .replace({"female": 1, "male": 0, "unknown": 2})
)
encoded_samples["family_id"] = LabelEncoder().fit_transform(
    encoded_samples["family_id"]
)


In [ ]:
#Using days to better interpret the distance between age groups
encoding_guide = {
    '1-2 weeks': 10,
    '4 weeks': 28,
    '8 weeks': 56,
    '4 months': 120,
    '5 months': 150,
    '6 months': 180,
    '9 months': 270,
    '11 months': 330,
    '14 months': 420,
}
encoded_samples["age_group_at_sample"] = encoded_samples["age_group_at_sample"].replace(encoding_guide)
encoded_samples["age_group_at_sample"] = encoded_samples["age_group_at_sample"].astype(int)
# consider in interpretation that the distances between the real age bins are not the same as our age groups

In [ ]:
leftovers = encoded_samples[encoded_samples["age_group_at_sample"].apply(lambda x: isinstance(x, str))]

if not leftovers.empty:

    print("Age group encoding:", leftovers["age_group_at_sample"].unique())

else:
    print("Fallback encoding not needed")


## Missing check


In [ ]:
missing_table = (
    encoded_samples.isna()
    .sum()
    .to_frame(name="missing_count")
    .assign(
        missing_percent=lambda df: (
            (df["missing_count"] / encoded_samples.shape[0] * 100).round(2)
        ),
    )
    .reset_index()
    .rename(columns={"index": "column"})
    .sort_values("missing_count", ascending=False)
    .query("missing_count != 0")
)

if len(missing_table) > 0:
    missing_table
else:
    print("No missing values detected.")

## Outlier check


In [ ]:
numeric_cols = encoded_samples.select_dtypes(include=[np.number]).columns

q1 = encoded_samples[numeric_cols].quantile(0.25)
q3 = encoded_samples[numeric_cols].quantile(0.75)
iqr = q3 - q1

lower_bounds = q1 - 1.5 * iqr
upper_bounds = q3 + 1.5 * iqr

outlier_mask = (
    (encoded_samples[numeric_cols] < lower_bounds)
    | (encoded_samples[numeric_cols] > upper_bounds)
)
outlier_counts = outlier_mask.sum()
outlier_percent = (outlier_counts / encoded_samples.shape[0] * 100).round(2)

outlier_table = (
    pd.DataFrame({
        "column": numeric_cols,
        "lower_bound": lower_bounds,
        "upper_bound": upper_bounds,
        "outlier_count": outlier_counts,
        "outlier_percent": outlier_percent,
    })
    .query("outlier_count > 0")
    .sort_values("outlier_percent", ascending=False)
    .reset_index(drop=True)
)

outlier_table

## Normalisation check


In [ ]:
normalized_samples = encoded_samples.copy()
print("Shapiro-Wilk Normality Test")

for column in numeric_cols:
    data_nona = normalized_samples[column].dropna()
    stat, p_value = stats.shapiro(data_nona)

    if p_value > 0.05:
        print(Fore.GREEN + f"{column}: Normally Distributed (p={p_value:.4f})")

    else:
        print(
            Fore.RED
            + f"{column}: Not Normally Distributed (p={p_value:.4f})"
        )

print(Style.RESET_ALL)

## Train-test split before pre-processing


In [ ]:
feature_cols = normalized_samples.columns.difference(["sample_id", "age_group_at_sample"]) # These variables will get removed from X

X = normalized_samples[feature_cols]
Y = normalized_samples["age_group_at_sample"]

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=MASTER_SEED)
train_indicies, test_indicies = next(gss.split(X, Y, groups=X['family_id']))
X_train_raw = X.iloc[train_indicies]
X_test_raw = X.iloc[test_indicies]
Y_train = Y.iloc[train_indicies]
Y_test = Y.iloc[test_indicies]

assert X_train_raw.shape[1] == X_test_raw.shape[1], "Feature columns do not match between train and test sets."
assert X_train_raw.shape[0] == Y_train.shape[0] and X_test_raw.shape[0] == Y_test.shape[0], "X and Y do not have the same length."

print("Train shape:", X_train_raw.shape, "| Test shape:", X_test_raw.shape)

## Normalising data using clr transformation


In [ ]:
X_train = clr_transform(X_train_raw)
X_test = clr_transform(X_test_raw)

In [ ]:
# To activate back the raw data without normalisation, uncomment the following:
# X_train = X_train_raw
# X_test = X_test_raw

In [ ]:
print("Shapiro-Wilk Normality Test (after Log Normalisation)\n")

for col in X_train.columns:

    # 1. Get the data for this column from both sets
    train_data = X_train[col].dropna()
    test_data = X_test[col].dropna()

    # 2. Run Shapiro test on both
    stat_train, p_train = stats.shapiro(train_data)
    stat_test, p_test = stats.shapiro(test_data)

    # 3. Determine status (Both must be > 0.05 to be truly "Normal")
    is_train_normal = p_train > 0.05
    is_test_normal = p_test > 0.05

    # 4. Print Logic
    # If both are Green
    if is_train_normal and is_test_normal:
        print(Fore.GREEN + f"✔ {col}: Normal (Train p={p_train:.3f}, Test p={p_test:.3f})")

    # If one is Red (Mixed results)
    elif is_train_normal or is_test_normal:
        print(Fore.YELLOW + f"⚠ {col}: Inconsistent (Train p={p_train:.3f}, Test p={p_test:.3f})")

    # If both are Red
    else:
        print(Fore.RED + f"✘ {col}: Not Normal (Train p={p_train:.3f}, Test p={p_test:.3f})")

print(Style.RESET_ALL)

# Model Training


In [ ]:
model_results = []

## Filtering for features at the genus level


In [ ]:
X_train_genus_raw = filter_features_by_level(X_train, max_level="Genus")
X_train_genus, _ = feature_selection_pipeline(X_train_genus_raw)
X_test_genus = X_test[X_train_genus.columns]

## Random Forest Regressor with Train/Test split (Genus)


In [ ]:
# Default starting parameters
current_params = {
    "prevalence_thresh": 0.05,
    "abundance_thresh": 1e-4,
    "variance_thresh": 1e-5,
    "corr_thresh": 0.9
}

# Maximum perturbation for each parameter
param_ranges = {
    "prevalence_thresh": 0.01,
    "abundance_thresh": 5e-5,
    "variance_thresh": 5e-6,
    "corr_thresh": 0.05
}

best_rmse = np.inf
best_r2 = -np.inf
results = []

n_iterations = 50  # you can increase this for a more thorough search

for i in range(n_iterations):
    # Random perturbation for all parameters
    new_params = {
        k: max(1e-9, current_params[k] + np.random.uniform(-param_ranges[k], param_ranges[k]))
        for k in current_params
    }
    # Keep corr_thresh <= 0.99
    if new_params["corr_thresh"] >= 1.0:
        new_params["corr_thresh"] = 0.99

    # Apply feature filtering
    X_train_filtered, removed_features = feature_selection_pipeline(
        X_train_genus_raw,
        prevalence_thresh=new_params["prevalence_thresh"],
        abundance_thresh=new_params["abundance_thresh"],
        variance_thresh=new_params["variance_thresh"],
        corr_thresh=new_params["corr_thresh"]
    )
    X_test_filtered = X_test[X_train_filtered.columns]

    # Train a quick model
    rf_temp = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    rf_temp.fit(X_train_filtered, Y_train)
    preds = rf_temp.predict(X_test_filtered)

    rmse = np.sqrt(mean_squared_error(Y_test, preds))
    r2 = r2_score(Y_test, preds)

    results.append({
        **new_params,
        "rmse": rmse,
        "r2": r2,
        "n_features": X_train_filtered.shape[1]
    })

    # Keep new params if performance improved
    if rmse < best_rmse:
        best_rmse = rmse
        best_r2 = r2
        current_params = new_params.copy()

    if (i+1) % 10 == 0:
        print(f"Iteration {i+1}: Best RMSE = {best_rmse:.2f}, Best R² = {best_r2:.3f}")

# Save all results for review
results_df = pd.DataFrame(results)
print("\nBest parameters found:")
print(current_params)
print(f"\nBest RMSE: {best_rmse:.2f}")
print(f"Best R²: {best_r2:.3f}")

## Random Forest Regressor with Train/Test split (Genus)


### Base model


In [ ]:
# Base model
rf_base = RandomForestRegressor(
    random_state=MASTER_SEED,
    n_jobs=-1,
    oob_score=True
)

rf_base.fit(X_train_genus, Y_train)
yhat_rf = rf_base.predict(X_test_genus)

print(f"Mean Squared Error: {mean_squared_error(Y_test, yhat_rf):.3f}")
print(f"Best CV RMSE: {np.sqrt(mean_squared_error(Y_test, yhat_rf)):.3f}")
print(f"R2 Score: {r2_score(Y_test, yhat_rf):.3f}")


### Search for the best model


In [ ]:
rf_results = random_forest_benchmark(
    X_train_genus,
    X_test_genus,
    Y_train,
    Y_test,
    label="Genus Level"
)

print(f"\nBest hyperparameters: {rf_results.best_params}")

In [ ]:
# Track Random Forest results
model_results.append({
    'model': 'Random Forest',
    'rmse': rf_results.rmse,
    'r2': rf_results.r2,
    'best_params': rf_results.best_params,
})

In [ ]:
print(f"Mean Squared Error: {rf_results.rmse**2:.3f}") # Squared because rmse is sqrt(mse)
print(f"Best CV RMSE: {rf_results.rmse:.3f}")
print(f"R2 Score: {rf_results.r2:.3f}")

best_rf_model = rf_results.model
yhat = rf_results.model.predict(X_test_genus)

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=Y_test, y=yhat, alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()],
         [Y_test.min(), Y_test.max()],
         color="red", linestyle="--")

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs Actual (Genus-level RF)")
plt.tight_layout()
plt.show()

In [ ]:
residuals = Y_test - yhat

plt.figure(figsize=(6,4))
sns.scatterplot(x=yhat, y=residuals, alpha=0.6)
plt.axhline(0, linestyle="--", color="red")

plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Predictions")
plt.tight_layout()
plt.show()

In [ ]:
importances = pd.Series(
    best_rf_model.feature_importances_,
    index=X_train_genus.columns
).sort_values(ascending=False)

top_n = 20

plt.figure(figsize=(8,6))
sns.barplot(
    x=importances.head(top_n),
    y=importances.head(top_n).index
)

plt.xlabel("Feature importance")
plt.ylabel("Genus")
plt.title(f"Top {top_n} most important genera")
plt.tight_layout()
plt.show()

In [ ]:
st = SuperTree(
    best_rf_model,
    X_train_genus,
    Y_train
)

st.show_tree(which_tree=0)

In [ ]:
# get predictions from each tree on the test set
all_tree_preds = np.array([tree.predict(X_test_genus) for tree in best_rf_model.estimators_])

# compute the mean prediction (Random Forest final prediction)
rf_pred = all_tree_preds.mean(axis=0)

# compute standard deviation per sample (uncertainty)
rf_std = all_tree_preds.std(axis=0)

plt.figure(figsize=(10,6))

# plot all tree predictions (semi-transparent lines)
for i in range(all_tree_preds.shape[0]):
    plt.plot(Y_test.values, all_tree_preds[i], 'o', color='lightgray', alpha=0.3)

# plot Random Forest mean prediction
plt.scatter(Y_test, rf_pred, color='blue', label='RF mean prediction', s=40)

plt.errorbar(Y_test, rf_pred, yerr=rf_std, fmt='o', color='red', alpha=0.5, label='±1 std across trees')

plt.plot([Y_test.min(), Y_test.max()],
         [Y_test.min(), Y_test.max()],
         color='black', linestyle='--', label='Perfect prediction')

plt.xlabel("Actual Age Group")
plt.ylabel("Predicted Age Group")
plt.title("Random Forest – Forest plot of tree predictions")
plt.legend()
plt.tight_layout()
plt.show()


# Alternative Models


## XGBoost Alternative


In [ ]:
# Base model
xgb_base = XGBRegressor(
    objective='reg:squarederror',
    random_state=MASTER_SEED,
    n_jobs=-1
)

X_train_clean = X_train_genus.copy()
X_train_clean.columns = [re.sub('[^A-Za-z0-9_]+', '', str(col)) for col in X_train_clean.columns]
X_test_clean = X_test_genus.copy()
X_test_clean.columns = [re.sub('[^A-Za-z0-9_]+', '', str(col)) for col in X_test_clean.columns]


xgb_base.fit(X_train_clean, Y_train)
yhat_xgb = xgb_base.predict(X_test_clean)

print(f"Mean Squared Error: {mean_squared_error(Y_test, yhat_xgb):.3f}")
print(f"Best CV RMSE: {np.sqrt(mean_squared_error(Y_test, yhat_xgb)):.3f}")
print(f"R2 Score: {r2_score(Y_test, yhat_xgb):.3f}")


### Best XGBoost Parameters Search


In [ ]:
xgboost_results = xgboost_benchmark(
    X_train_genus,
    X_test_genus,
    Y_train,
    Y_test,
    label="Genus Level"
)
print(f"Best hyperparameters: {xgboost_results.best_params}")

In [ ]:
# Track XGBoost results
model_results.append({
    'model': 'XGBoost',
    'rmse': xgboost_results.rmse,
    'r2': xgboost_results.r2,
    'best_params': xgboost_results.best_params,
})


## LightGBM


In [ ]:
lgb_results = lightgbm_benchmark(
    X_train_genus,
    X_test_genus,
    Y_train,
    Y_test,
    label="Genus Level"
)

print(f"\nBest hyperparameters: {lgb_results.best_params}")

In [ ]:
print(f"Mean Squared Error: {lgb_results.rmse**2:.3f}")
print(f"Best CV RMSE: {lgb_results.rmse:.3f}")
print(f"R2 Score: {lgb_results.r2:.3f}")

best_lgb_model = lgb_results.model
yhat_lgb = lgb_results.model.predict(X_test_genus)

In [ ]:
# Track LightGBM results
model_results.append({
    'model': 'LightGBM',
    'rmse': lgb_results.rmse,
    'r2': lgb_results.r2,
    'best_params': lgb_results.best_params
})

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=Y_test, y=yhat_lgb, alpha=0.6)
plt.plot([Y_test.min(), Y_test.max()],
         [Y_test.min(), Y_test.max()],
         color="red", linestyle="--")

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs Actual (Genus-level LightGBM)")
plt.tight_layout()
plt.show()

In [ ]:
importances_lgb = pd.Series(
    best_lgb_model.feature_importances_,
    index=X_train_genus.columns
).sort_values(ascending=False)

top_n = 20

plt.figure(figsize=(8,6))
sns.barplot(
    x=importances_lgb.head(top_n),
    y=importances_lgb.head(top_n).index
)

plt.xlabel("Feature importance")
plt.ylabel("Genus")
plt.title(f"Top {top_n} most important genera (LightGBM)")
plt.tight_layout()
plt.show()